In [65]:
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import talib
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%maplotlib inline


In [66]:
df = pd.read_csv('/Users/zed/AI_Lab/DoubleEnsembleML/Data/data_factor/BTC_fac.csv',index_col=0)

In [67]:
df

,High,Low,Close,Volume,H-L,3day MA,10day MA,30day MA,Std_dev,RSI,...,Middleband,Lowerband,ADX,Last 1 Price,Last 2 Price,Last 3 Price,Last 4 Price,Last 5 Price,Target_value,Target_tag
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-08,667.98953,623.00000,636.00880,4.199144,44.98953,661.298203,624.213692,637.954088,22.648563,49.762118,...,660.100682,619.585701,38.875402,636.20048,666.70413,680.99000,680.60000,666.94350,0.018751,1
2014-03-09,658.00000,629.05259,647.93490,2.017482,28.94741,646.304470,629.087439,633.567420,19.787699,52.986768,...,653.567662,618.170351,37.276340,636.00880,636.20048,666.70413,680.99000,680.60000,-0.009043,0
2014-03-10,684.50000,630.00000,642.07570,6.307648,54.50000,640.048060,634.232300,631.361182,12.682942,51.171375,...,645.784802,623.096865,35.214074,647.93490,636.00880,636.20048,666.70413,680.99000,-0.001675,0
2014-03-11,665.93423,560.00000,641.00000,18.486550,105.93423,642.006467,641.045751,629.453620,4.913341,50.811813,...,640.643976,631.854725,34.377755,642.07570,647.93490,636.00880,636.20048,666.70413,-0.013555,0
2014-03-12,660.00000,550.27500,632.31094,8.316095,109.72500,643.670200,647.645751,627.853620,5.983995,47.762068,...,639.866068,629.161572,33.732704,641.00000,642.07570,647.93490,636.00880,636.20048,0.033349,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-19,57594.70000,54228.90000,55720.00000,6649.986168,3365.80000,59260.000000,60524.450000,57960.256667,3277.850945,35.326815,...,59348.600000,53485.001973,22.866719,56239.80000,60085.20000,61455.00000,63243.00000,62969.00000,0.013999,1
2021-04-20,57110.40000,53388.30000,56500.00000,7804.231919,3722.10000,57348.333333,60285.450000,57881.203333,2589.970467,39.522061,...,58000.000000,53366.919981,23.300178,55720.00000,56239.80000,60085.20000,61455.00000,63243.00000,-0.047519,0
2021-04-21,56805.50000,53600.00000,53815.20000,4788.013532,3205.50000,56153.266667,59957.890000,57851.923333,2276.730403,31.587568,...,56472.040000,52399.300842,23.702676,56500.00000,55720.00000,56239.80000,60085.20000,61455.00000,-0.038747,0


In [68]:
df.describe()

,High,Low,Close,Volume,H-L,3day MA,10day MA,30day MA,Std_dev,RSI,...,Middleband,Lowerband,ADX,Last 1 Price,Last 2 Price,Last 3 Price,Last 4 Price,Last 5 Price,Target_value,Target_tag
count,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,...,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000,2557.000000
mean,6636.219081,6214.205968,6451.119386,3871.879949,422.013114,6411.113090,6333.880817,6104.753951,244.140469,54.207836,...,6410.646355,5973.914608,26.559681,6431.353059,6411.383056,6390.603154,6368.773119,6347.242788,inf,0.535784
std,10453.749359,9737.781918,10149.416527,4351.137144,877.505598,10063.449300,9870.208962,9249.607344,520.380308,15.877072,...,10055.667233,9380.569732,12.276872,10111.419467,10072.268948,10029.150578,9980.669569,9933.435844,NaN,0.498815
min,0.000000,0.000000,0.000000,0.000000,0.000000,158.335503,212.667667,225.700348,0.530946,4.487266,...,204.672368,-194.329929,3.294964,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
25%,505.000000,475.000000,489.724430,175.521807,17.002000,479.537803,467.002904,477.391989,11.003148,43.944891,...,468.765034,442.423482,17.187235,489.724430,489.724430,489.724430,489.724430,489.724430,-0.013932,0.000000
50%,3849.000000,3606.100000,3719.000000,2885.956979,134.400000,3752.833333,3773.300000,3691.760000,78.120996,52.597962,...,3768.940000,3516.030996,25.547971,3716.400000,3700.100000,3694.300000,3691.900000,3683.600000,0.001584,1.000000
75%,8932.500000,8410.000000,8715.500000,5762.249495,426.700000,8684.966667,8626.130000,8652.786667,243.644099,64.361467,...,8659.380000,8118.373241,34.191375,8715.100000,8711.500000,8705.800000,8698.200000,8694.000000,0.018826,1.000000
max,64900.000000,62050.000000,63587.200000,45110.873425,12565.600000,63266.400000,60704.160000,58022.323333,7517.225453,97.427121,...,62267.880000,59643.700218,60.756691,63587.200000,63587.200000,63587.200000,63587.200000,63587.200000,inf,1.000000


In [69]:
y = df["Target_tag"]
X = df.drop(["Target_value",'Target_tag'],axis=1 )


In [70]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers)
        
        self.fc = nn.Linear(hidden_size, num_classes)
        

    def forward(self, x):
        # 不手动指定 h 和 c 的话，默认就是 0
#         h_0 = torch.zeros(
#             self.num_layers, x.size(0), self.hidden_size)
        
#         c_0 = torch.zeros(
#             self.num_layers, x.size(0), self.hidden_size)
        
        # Propagate input through LSTM
#         ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        ula, (h_out, _) = self.lstm(x)
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out
def sliding_windows(data, label, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = label[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [71]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
seq_length = 15

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

Y = y.values.reshape(-1,1)

Y = y_scaler.fit_transform(Y)

x, y = sliding_windows(X, Y,15)

In [72]:
y_train,y_test = y[:int(y.shape[0]*0.8)],y[int(y.shape[0]*0.8):]
x_train,x_test = x[:int(x.shape[0]*0.8)],x[int(x.shape[0]*0.8):]

# lstm: seq, batch, feature
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataX = torch.Tensor(x.transpose(1,0,2))
dataY = torch.Tensor(y)
trainX = torch.Tensor(x_train.transpose(1,0,2))
trainY = torch.Tensor(y_train)
testX = torch.Tensor(x_test.transpose(1,0,2))
testY = torch.Tensor(y_test)
trainX.shape, trainY.shape

(torch.Size([15, 2032, 20]), torch.Size([2032, 1]))

In [73]:
num_epochs = 10
learning_rate = 0.0001

input_size = X.shape[1] # The number of expected features in the input x
hidden_size = 30 # The number of features in the hidden state h
num_layers = 1 # Number of recurrent layers.

num_classes = 1 # output

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
#optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model
lstm.train()
lstm.to(device)
trainX = trainX.to(device)
for epoch in tqdm(range(num_epochs)):
    optimizer.zero_grad()
    print(outputs.shape)
    outputs = lstm(trainX)
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch%10 == 0 :
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

 10%|█         | 1/10 [00:00<00:01,  7.42it/s]

torch.Size([2032, 1])
Epoch: 0, loss: 0.63446
torch.Size([2032, 1])


 30%|███       | 3/10 [00:00<00:00,  8.56it/s]

torch.Size([2032, 1])
torch.Size([2032, 1])


 50%|█████     | 5/10 [00:00<00:00,  8.88it/s]

...

 70%|███████   | 7/10 [00:00<00:00,  8.84it/s]

torch.Size([2032, 1])
torch.Size([2032, 1])


 90%|█████████ | 9/10 [00:01<00:00,  7.05it/s]

torch.Size([2032, 1])
torch.Size([2032, 1])


100%|██████████| 10/10 [00:01<00:00,  7.50it/s]


In [75]:
import plotly.graph_objects as go
# from IPython.display import HTML
import chart_studio.plotly as py
lstm.eval()
lstm.to(torch.device('cpu'))
with torch.no_grad():
    dataY_pred = lstm(dataX)

dataY_pred = dataY_pred.data.numpy()
dataY_truth = dataY.data.numpy()

dataY_pred = y_scaler.inverse_transform(dataY_pred)
dataY_truth = y_scaler.inverse_transform(dataY_truth)


fig = go.Figure(go.Scatter(y=dataY_truth.flatten(),name='Ground Truth'))
fig.add_trace(go.Scatter(y=dataY_pred.flatten(),name='Predicted'))

fig.update_layout(
    shapes = [dict(
        x0=len(x_train), x1=len(x_train), y0=0, y1=1, xref='x', yref='paper',
        line_width=2)], #在图上划分训练集和测试集
    xaxis_rangeslider_visible=True,
)
py.iplot(fig, filename="stock-result")

PlotlyRequestError: Authentication credentials were not provided.

PlotlyRequestError: Authentication credentials were not provided.